In [3]:
%matplotlib inline
import astropy.io.fits as pyfits
import numpy as np
from astropy.utils.data import get_pkg_data_filename
import subprocess as sp
import scipy.ndimage as sci_nd
import glob
import matplotlib.pyplot as mpl
from astropy.table import Table
import sys
import os
from astropy.io import ascii


In [4]:
#Bruno's github codes

def get_fixpars_default():
    r""" Returns the default dictionary containing the information on whether
    or not to fix any parameter of the fit. By default, all parameters are
    not fixed.
    Parameters
    ----------
    Returns
    -------
    fixpars : dict
        A dictionary for each of the sersic parameters setting the fix/free key.
    References
    ----------
    Examples
    --------
    """
    return {'x':1,'y':1,'m':1,'re':1,'n':1,'q':1,'pa':1,'sky':1}


def write_object(model,x,y,m,re,n,ba,pa,num,fixpars=None):
    r""" Returns a string object containing a general description for a galaxy
    profile model in GALFIT, with the input parameters as first guesses.
    Parameters
    ----------
    model : str
    Returns
    -------
    References
    ----------
    Examples
    --------
    """
    if fixpars is None:
        fixpars=get_fixpars_default()

    objString = ""
    objString += "#Object number: %i\n"%(num)
    objString += " 0) %s             # Object type\n"%(model)
    objString += " 1) %6.4f %6.4f  %i %i    # position x, y        [pixel]\n"%(x,y,fixpars['x'],fixpars['y'])
    objString += " 3) %4.4f      %i       # total magnitude\n"%(m,fixpars['m'])
    objString += " 4) %4.4f       %i       #     R_e              [Pixels]\n"%(re,fixpars['re'])
    objString += " 5) %4.4f       %i       # Sersic exponent (deVauc=4, expdisk=1)\n"%(n,fixpars['n'])
    objString += " 9) %4.4f       %i       # axis ratio (b/a)   \n"%(ba,fixpars['q'])
    objString += "10) %4.4f       %i       # position angle (PA)  [Degrees: Up=0, Left=90]\n"%(pa,fixpars['pa'])
    objString += " Z) 0                  #  Skip this model in output image?  (yes=1, no=0)\n"
    objString += " \n"
    return objString


def input_file(f,modelsString,magzpt,sky,x_range,y_range,sconvbox,pixscale,imgname='input.fits',outname="output.fits",psfname='none',maskname="none",signame='none',fixpars=None):
    r""" Writes an input galfit file with the provided parameters.
    Parameters
    ----------
    f : file pointer
        Python file pointer to be written
    modelsString : str
        A single string containing the starting parameters of the models to be
        used. Model strings can be generated by the write_object function.
    magzpt : float
        Magnitude zeropoint of the image data. Needed for correct model
        magnitudes.
    sky : float
        Initial value for sky background (assumed to be constant)
    x_range : tuple, int
        X-axis boundaries to be used in image fitting (useful to fit a subset
        of a larger image).
    y_range : tuple, int
        Same as z_range, but for y-axis boundaries.
    sconvbox : int
        Size of the convolution box to be applied to the model image (should be
        at least the size of the PSF image).
    pixscale : float
        Pixel scale (in arcsecond/pixel) of the input image data
    imgname : str, optional
        Name of the image with the input data (default - input.fits)
    outname : str, optional
        Name of the output image block to be created by GALFIT
        (default - output.fits)
    psfname : str, optional
        Name of the psf fits file, to be used for model convolution
        (default - none, no psf used)
    maskname : str, optional
        Name of the mask file to be used in the fit. It mask bad pixel values
        and others (e.g. neighbor objects) that are ignored by the fitting
        algorithm (default - none, no mask used).
    signame : str, optional
        Error image corresponding to the input data.
        (default - none, sigma image created internally).
    fixpars : dict
        Dictionary which controls if parameters are fixed or not for the fit.
        This is used here to control wether the sky background is fixed or a
        free parameter for the fit.
    Returns
    -------
        None
        It writes to the file "f" and returns nothing.
    References
    ----------
    Examples
    --------
    """
    if fixpars is None:
        fixpars=get_fixpars_default()

    assert len(x_range)==len(y_range)==2,"x_range,y_range must have two elements"
    assert x_range[1]>x_range[0],"x_range must be sorted in ascendent order"
    assert y_range[1]>y_range[0],"y_range must be sorted in ascendent order"

    f.write("================================================================================\n")
    f.write("# IMAGE and GALFIT CONTROL PARAMETERS\n")
    f.write("A) %s         # Input data image (FITS file)\n"%imgname)
    f.write("B) %s        # Output data image block\n"%outname)
    f.write("C) %s                # Sigma image name (made from data if blank or 'none' \n"%signame)
    f.write("D) %s         # Input PSF image and (optional) diffusion kernel\n"%psfname)
    f.write("E) 1                   # PSF fine sampling factor relative to data \n")
    f.write("F) %s                # Bad pixel mask (FITS image or ASCII coord list)\n"%maskname)
    f.write("G) none                # File with parameter constraints (ASCII file) \n")
    f.write("H) %i    %i   %i    %i # Image region to fit (xmin xmax ymin ymax)\n"%(x_range[0],x_range[1],y_range[0],y_range[1]))
    f.write("I) %i    %i          # Size of the convolution box (x y)\n"%(sconvbox,sconvbox))
    f.write("J) %7.5f             # Magnitude photometric zeropoint \n"%magzpt)
    f.write("K) %.3f %.3f        # Plate scale (dx dy)   [arcsec per pixel]\n"%(pixscale,pixscale))
    f.write("O) regular             # Display type (regular, curses, both)\n")
    f.write("P) 0                   # Options: 0=normal run; 1,2=make model/imgblock and quit\n")
    f.write("\n")
    f.write("# INITIAL FITTING PARAMETERS\n")
    f.write("#\n")
    f.write("#For object type, the allowed functions are:\n")
    f.write("#nuker, sersic, expdisk, devauc, king, psf, gaussian, moffat,\n")
    f.write("#ferrer, and sky.\n")
    f.write("#\n")
    f.write("#Hidden parameters will only appear when theyre specified:\n")
    f.write("#C0 (diskyness/boxyness),\n")
    f.write("#Fn (n=integer, Azimuthal Fourier Modes).\n")
    f.write("#R0-R10 (PA rotation, for creating spiral structures).\n")
    f.write("#\n")
    f.write("# ------------------------------------------------------------------------------\n")
    f.write("#  par)    par value(s)    fit toggle(s)   parameter description\n")
    f.write("# ------------------------------------------------------------------------------\n")
    f.write("\n")

    f.write(modelsString)

    f.write("# Object: Sky\n")
    f.write(" 0) sky                    #  object type\n")
    f.write(" 1) %7.4f      %i          #  sky background at center of fitting region [ADUs]\n"%(sky,fixpars['sky']))
    f.write(" 2) 0.0000      0          #  dsky/dx (sky gradient in x)\n")
    f.write(" 3) 0.0000      0          #  dsky/dy (sky gradient in y)\n")
    f.write(" Z) 0                      #  output option (0 = resid., 1 = Dont subtract)")
    f.close()
    return


#amended code

def select_object_segmap(segmap,index):
    Regions = np.zeros_like(segmap)
    if index not in segmap:
        raise ValueError('index not in segmentation map')
    Regions[segmap==index]=index
    return Regions

In [5]:
masterdata = ascii.read('/data1/rowland/mastertable')
masterdata

ACT-CL,SMH,RAJ2000,DEJ2000,imag,z,e_z,rcc,MSF,Simbad,recno,NUMBER_F105,MAG_AUTO_F105,MAGERR_AUTO_F105,KRON_RADIUS_F105,ISOAREA_IMAGE_F105,XPEAK_IMAGE_F105,YPEAK_IMAGE_F105,X_IMAGE_F105,Y_IMAGE_F105,ALPHA_J2000_F105,DELTA_J2000_F105,A_IMAGE_F105,THETA_IMAGE_F105,MU_MAX_F105,ELLIPTICITY_F105,CLASS_STAR_F105,FLUX_RADIUS_F105,Separation_1,NUMBER_F125,MAG_AUTO_F125,MAGERR_AUTO_F125,KRON_RADIUS_F125,ISOAREA_IMAGE_F125,XPEAK_IMAGE_F125,YPEAK_IMAGE_F125,X_IMAGE_F125,Y_IMAGE_F125,ALPHA_J2000_F125,DELTA_J2000_F125,A_IMAGE_F125,THETA_IMAGE_F125,MU_MAX_F125,ELLIPTICITY_F125,CLASS_STAR_F125,FLUX_RADIUS_F125,Separation_1a,NUMBER_F140,MAG_AUTO_F140,MAGERR_AUTO_F140,KRON_RADIUS_F140,ISOAREA_IMAGE_F140,XPEAK_IMAGE_F140,YPEAK_IMAGE_F140,X_IMAGE_F140,Y_IMAGE_F140,ALPHA_J2000_F140,DELTA_J2000_F140,A_IMAGE_F140,THETA_IMAGE_F140,MU_MAX_F140,ELLIPTICITY_F140,CLASS_STAR_F140,FLUX_RADIUS_F140,Separation_1b,NUMBER_F160,MAG_AUTO_F160,MAGERR_AUTO_F160,KRON_RADIUS_F160,ISOAREA_IMAGE_F160,XPEAK_IMAGE_F160,YPEAK_IMAGE_F160,X_IMAGE_F160,Y_IMAGE_F160,ALPHA_J2000_F160,DELTA_J2000_F160,A_IMAGE_F160,THETA_IMAGE_F160,MU_MAX_F160,ELLIPTICITY_F160,CLASS_STAR_F160,FLUX_RADIUS_F160,Separation_1c,NUMBER_F435,MAG_AUTO_F435,MAGERR_AUTO_F435,KRON_RADIUS_F435,ISOAREA_IMAGE_F435,XPEAK_IMAGE_F435,YPEAK_IMAGE_F435,X_IMAGE_F435,Y_IMAGE_F435,ALPHA_J2000_F435,DELTA_J2000_F435,A_IMAGE_F435,THETA_IMAGE_F435,MU_MAX_F435,ELLIPTICITY_F435,CLASS_STAR_F435,FLUX_RADIUS_F435,Separation_1d,NUMBER_F606,MAG_AUTO_F606,MAGERR_AUTO_F606,KRON_RADIUS_F606,ISOAREA_IMAGE_F606,XPEAK_IMAGE_F606,YPEAK_IMAGE_F606,X_IMAGE_F606,Y_IMAGE_F606,ALPHA_J2000_F606,DELTA_J2000_F606,A_IMAGE_F606,THETA_IMAGE_F606,MU_MAX_F606,ELLIPTICITY_F606,CLASS_STAR_F606,FLUX_RADIUS_F606,Separation_1e,NUMBER_F625,MAG_AUTO_F625,MAGERR_AUTO_F625,KRON_RADIUS_F625,ISOAREA_IMAGE_F625,XPEAK_IMAGE_F625,YPEAK_IMAGE_F625,X_IMAGE_F625,Y_IMAGE_F625,ALPHA_J2000_F625,DELTA_J2000_F625,A_IMAGE_F625,THETA_IMAGE_F625,MU_MAX_F625,ELLIPTICITY_F625,CLASS_STAR_F625,FLUX_RADIUS_F625,Separation_1f,NUMBER_F775,MAG_AUTO_F775,MAGERR_AUTO_F775,KRON_RADIUS_F775,ISOAREA_IMAGE_F775,XPEAK_IMAGE_F775,YPEAK_IMAGE_F775,X_IMAGE_F775,Y_IMAGE_F775,ALPHA_J2000_F775,DELTA_J2000_F775,A_IMAGE_F775,THETA_IMAGE_F775,MU_MAX_F775,ELLIPTICITY_F775,CLASS_STAR_F775,FLUX_RADIUS_F775,Separation_1g,NUMBER_F814,MAG_AUTO_F814,MAGERR_AUTO_F814,KRON_RADIUS_F814,ISOAREA_IMAGE_F814,XPEAK_IMAGE_F814,YPEAK_IMAGE_F814,X_IMAGE_F814,Y_IMAGE_F814,ALPHA_J2000_F814,DELTA_J2000_F814,A_IMAGE_F814,THETA_IMAGE_F814,MU_MAX_F814,ELLIPTICITY_F814,CLASS_STAR_F814,FLUX_RADIUS_F814,Separation_1h,NUMBER_F850,MAG_AUTO_F850,MAGERR_AUTO_F850,KRON_RADIUS_F850,ISOAREA_IMAGE_F850,XPEAK_IMAGE_F850,YPEAK_IMAGE_F850,X_IMAGE_F850,Y_IMAGE_F850,ALPHA_J2000_F850,DELTA_J2000_F850,A_IMAGE_F850,THETA_IMAGE_F850,MU_MAX_F850,ELLIPTICITY_F850,CLASS_STAR_F850,FLUX_RADIUS_F850,Separation
str10,str18,float64,float64,float64,float64,float64,float64,str16,str6,int64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float6

In [4]:
# for count,i in enumerate(masterdata['NUMBER_F105']):
#     print(count,i)

In [ ]:
#testing



# #segmap = segmap_file[0].data+0 #PARAMETER
# collected_regions = []
# master_filters = ["F105","F125","F140","F160","F435","F606","F625","F775","F814","F850"]



# for filter in master_filters:
#     if filter is not 'F850': #need this b/c table colnames arent full filternames
#         filtername = filter + 'W'
#     else:
#         filtername = filter + 'LP'
#     segmap = pyfits.open('/data1/rowland/segmaps_check/{}_check.fits'.format(filtername))[0].data
#     print(segmap.shape)
#     #print(filtername)
#     for detection in masterdata: 

#         #print(detection)
#         xc = detection["X_IMAGE_{}".format(filter)] #in pix #PARAMETER
#         yc = detection["Y_IMAGE_{}".format(filter)] #in pix #PARAMETER
#         #print(xc,yc)
#         pixscale = 0.06 #PARAMETER
#         index = detection['NUMBER_{}'.format(filter)]
#         #print(index.dtype)
#         radius=0.5 ##DEFAULT value for reference #PARAMETER
#         if index.dtype == 'int64':
#             #print('chose this path')
#             each_region = select_object_segmap(segmap,index) #indexing so have matching value so subtracting =0
#         else:
#             #print('chose that path')
#             pass
#         collected_regions.append(each_region)
#     break
# collected_regions = np.array(collected_regions)
# print('collected_regions: type-',type(collected_regions), 'shape-',collected_regions.shape)    
# final_regions = np.sum(collected_regions, axis=0) #COLLECTING ALL OUTPUTS TO ONE MASK
# print('shape of final regions:',final_regions.shape)

(4824, 4538)


In [4]:
#think gotta make a fcn b/c cant save indiv filter collected regions another way


# def regions(filtername):
#     collected_regions = []
#     masterdata = ascii.read('/data1/rowland/mastertable')
#     segmap = pyfits.open('/data1/rowland/segmaps_check/{}_check.fits'.format(filtername))[0].data
#         #print(segmap.shape)
#         #print(filtername)
#     for detection in masterdata: 
#         if filtername is not 'F850LP': #need this b/c table colnames arent full filternames
#             filter = filtername[:-1]
#             #print('chose this path:',filter)
#         else:
#             filter = filtername[:-2]
#             #print('chose that path:',filter)
#         #print(detection)
#         xc = detection["X_IMAGE_{}".format(filter)] #in pix #PARAMETER
#         yc = detection["Y_IMAGE_{}".format(filter)] #in pix #PARAMETER
#         #print(xc,yc)
#         pixscale = 0.06 #PARAMETER
#         index = detection['NUMBER_{}'.format(filter)]
#         #print(index.dtype)
#         radius=0.5 ##DEFAULT value for reference #PARAMETER
#         if index.dtype == 'int64':
#             #print('chose this path')
#             each_region = select_object_segmap(segmap,index) #indexing so have matching value so subtracting =0
#         else:
#             #print('chose that path')
#             pass
#     collected_regions.append(each_region)
#     return(collected_regions)


# 1. Made MASKs on mac

In [5]:
#runs through each filter to get regions for each

# filters = ["F105W","F125W","F140W","F160W","F435W","F606W","F625W","F775W","F814W","F850LP"]

# allregions = []
# for filter in filters:
#     eachregion = regions(filter)
#     allregions.append(eachregion)
# np.shape(allregions)

(10, 1)

In [7]:
#how to index/separate each filters info


#np.shape(allregions[3][0]) #change first index to go through each region

(4824, 4538)

In [35]:
#testing how to sum these to create mask PER FILTER

# collected_regions = np.array(collected_regions)
# print('collected_regions: type-',type(collected_regions), 'shape-',collected_regions.shape)    
# final_regions = np.sum(collected_regions, axis=0) #COLLECTING ALL OUTPUTS TO ONE MASK
# print('shape of final regions:',final_regions.shape)

# for arr in allregions:
#     print(np.shape(arr))

(1, 4824, 4538)
(1, 4824, 4538)
(1, 4824, 4538)
(1, 4824, 4538)
(1, 4256, 4397)
(1, 8473, 8201)
(1, 8473, 8201)
(1, 8473, 8201)
(1, 8473, 8201)
(1, 8473, 8201)


In [6]:
# finalregions=[]

# for arr in allregions:
#     finalregion = np.sum(arr,axis=0)
#     finalregions.append(finalregion)
# np.shape(finalregions[0])

(4824, 4538)

In [39]:
# masks=[]
# for count,filter in enumerate(filters):
#     segmap = pyfits.open('/data1/rowland/segmaps_check/{}_check.fits'.format(filter))[0].data
#     mask = segmap - finalregions[count]
#     masks.append(mask)
# np.shape(masks) #one mask per filter

(10,)

In [40]:
# finalmasks = []

# for count,filter in enumerate(filters):
#     masks[count][np.where(masks[count] > 0)]=1
#     image_data = pyfits.open('/data1/rowland/elgordo_{}.fits'.format(filter))[0].data
#     masks[count][image_data==0]=1
#     finalmasks.append(masks)
# np.shape(finalmasks)

(10, 10)

In [44]:
# image_dataF105 = pyfits.open('/data1/rowland/elgordo_F105W.fits')[0].data
# segmapF105 = pyfits.open('/data1/rowland/segmaps_check/F105W_check.fits')[0].data

# x = masterdata['X_IMAGE_F105'][0]
# y = masterdata['Y_IMAGE_F105'][0]

In [4]:


# fig, axis = mpl.subplots(1,4,figsize=(18,8),sharex=True,sharey=True)
# axis[0].imshow(image_dataF105,vmin = -0.01,vmax = 0.1)
# axis[0].set_title('hubble data')
# axis[1].imshow(segmapF105,vmin=0,vmax=1)
# axis[1].set_title('segmap')
# axis[2].imshow(finalregions[0][0],vmin=0, vmax=1)
# axis[2].set_title('detected regions')
# axis[3].imshow(finalmasks[0],vmin=0,vmax=1)
# axis[3].set_title('mask')

# for ax in axis:
#     #ax.plot(xc, yc, "+",ms=20,mew=3) #plots last run detected object
    
#     w = 250
#     ax.set_xlim(x-w,x+w)
#     ax.set_ylim(y-w,y+w)

# 2. Writing GalFit Input files for detections (each galaxy in cluster)

In [6]:
#collecting data of detections i want galfit to fit (ie cluster galaxies)


model = 'sersic' #from bruno
modelsList = [] #FOR INDIVIDUAL DETECTIONS
object_numbers = []
xs = []
ys = []
modelsString = [] #FOR ALL DETECTIONS 

def modString(filter):
    for detection in masterdata: #to only run on subset for testing
        ## DEFINING PARAMETERS
        try:
            num = detection['NUMBER_{}'.format(filter)] #object detection number from SEx list
            ba = 1 - detection['ELLIPTICITY_{}'.format(filter)] 
            pa = 90 - detection['THETA_IMAGE_{}'.format(filter)] 
            x = detection['X_IMAGE_{}'.format(filter)] 
            y = detection['Y_IMAGE_{}'.format(filter)]
            re = detection['FLUX_RADIUS_{}'.format(filter)]
            m = detection['MAG_AUTO_{}'.format(filter)]
            n = 4     #this is sersic index, 4 b/c mostly expecting distinct cores & extended disks
            list_of_strings = write_object(model,x,y,m,re,n,ba,pa,num,fixpars=None) 
            object_numbers.append(num)
        except:
            pass
        xs.append(x) #so i can slice these the same as modelsList in next cell
        ys.append(y)
        modelsList.append(list_of_strings) #FOR INDIVIDUAL DETECTIONS INPUT FILES
        modelsString = ''.join(modelsList) #FOR ALL DETECTIONS INPUT FILE
    return modelsList, modelsString

In [7]:
tfilters = ["F105","F125","F140","F160","F435","F606","F625","F775","F814","F850"]

lists =[]
strings =[]
for filter in tfilters:
    l,s = modString(filter)
    lists.append(l)
    strings.append(s)
np.shape(lists),np.shape(strings)

((10, 890), (10,))

In [14]:
#lists[0]

In [15]:
#strings[0]

In [20]:
#this makes a galfit input file for entire filter's detections 
filters = ["F105W","F125W","F140W","F160W","F435W","F606W","F625W","F775W","F814W","F850LP"]
zeropoints = {"F435W":25.665,"F606W":26.493,"F814W":25.97,"F105W":26.2687,"F125W":26.2303,"F140W":24.4524,\
              "F160W":25.9463, "F625W":24.85714771, "F775W":25.66195995, "F850LP":25.89943399}


for count,filtername in enumerate(filters):
    f_all = open("/data1/rowland/{}/input_{}_ALL.galfit".format(filtername,filtername), "w") #created a blank file to write to
    magzpt = zeropoints[filtername] #from zeropoints for this filter
    sky = 0 #b/c images already background subtracted
    if filtername == 'F105W' or 'F125W' or 'F160W' or 'F140W':
        x_range =  (0,4538) #from image_file[0].data.shape (x,y)
        y_range = (0,4824)  #from image_file[0].data.shape (x,y)
    elif filtername == 'F435W':
        x_range =  (0,4397) #from image_file[0].data.shape (x,y)
        y_range = (0,4256)
    else:
        x_range =  (0,8201) #from image_file[0].data.shape (x,y)
        y_range = (0,8473) 
    sconvbox = 100 #(4824, 4538)
    pixscale = 0.06
    strings_forgalfit = input_file(f_all,strings[count],magzpt,sky,x_range,y_range,sconvbox,pixscale,\
                                   imgname='elgordo_{}.fits'.format(filtername),\
                                   outname="output_{}_ALL.fits".format(filtername),\
                                   psfname='psf.fits',\
                                   maskname='dilated_mask_{}.fits'.format(filtername),\
                                   signame='none',fixpars=None)

In [12]:
np.shape(lists[0][0:2])

(2,)

# RUN GALFIT

# 3. To view all images per INDIVIDUAL galaxy: uncomment below

In [ ]:
for count, item in enumerate(lists[0][0:2], 0):
    print(count)
    image = get_pkg_data_filename('/data1/rowland/F105W/output_F105W_short.fits')
    image_1 = pyfits.getdata(image, ext=1) #original
    image_2 = pyfits.getdata(image, ext=2) #model
    image_3 = pyfits.getdata(image, ext=3) #residuals
    image_4 = image_3 / image_1 #residuals divided by originals
    
    #plotting output images
    f, (ax1,ax2,ax3,ax4) = mpl.subplots(1,4,sharey=True,figsize=(35,35))
    ax1.imshow(image_1,vmin=0,vmax=1)
    ax1.title.set_text('{}_First image/ original data'.format(count))
    ax2.imshow(image_2,vmin=0,vmax=1)
    ax2.title.set_text('{}_Second image/ FitModel'.format(count))
    ax3.imshow(image_3,vmin=0,vmax=1)
    ax3.title.set_text('{}_Third image/ Residuals'.format(count))
    ax4.imshow(image_4,vmin=0,vmax=1)
    ax4.title.set_text('{}_Fourth image -- Residuals/Original'.format(count))

0


/software/local/lib64/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys


1
